In [1]:
# Initialize a flag to track if all steps pass
all_steps_passed = True

# Step 1: Import the necessary package
try:
    import skl2onnx
    from skl2onnx import convert_sklearn
    from skl2onnx.common.data_types import FloatTensorType
    print("Step 1: Importing skl2onnx and necessary modules passed.")
except ImportError as e:
    all_steps_passed = False
    print(f"Step 1 failed: {e}")

# Step 2: Train a simple model using scikit-learn
try:
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier

    # Load dataset and split it
    iris = load_iris()
    X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

    # Train a RandomForestClassifier
    model = RandomForestClassifier(n_estimators=10)
    model.fit(X_train, y_train)

    assert model.score(X_test, y_test) > 0.8, "Model accuracy is too low."

    print("Step 2: Training a scikit-learn RandomForestClassifier passed.")
except AssertionError as e:
    all_steps_passed = False
    print(f"Step 2 failed: {e}")
except Exception as e:
    all_steps_passed = False
    print(f"Step 2 failed: {e}")

# Step 3: Convert the trained model to ONNX format
try:
    # Convert the model to ONNX format
    initial_type = [('float_input', FloatTensorType([None, 4]))]
    onnx_model = convert_sklearn(model, initial_types=initial_type)
    
    assert onnx_model is not None, "Conversion to ONNX failed."
    
    print("Step 3: Converting the model to ONNX format passed.")
except AssertionError as e:
    all_steps_passed = False
    print(f"Step 3 failed: {e}")
except Exception as e:
    all_steps_passed = False
    print(f"Step 3 failed: {e}")

# Step 4: Save the ONNX model to a file
try:
    import onnx
    
    onnx_model_path = "rf_model.onnx"
    with open(onnx_model_path, "wb") as f:
        f.write(onnx_model.SerializeToString())
    
    # Load the model back to ensure it's saved correctly
    loaded_model = onnx.load(onnx_model_path)
    onnx.checker.check_model(loaded_model)
    
    print("Step 4: Saving and validating the ONNX model passed.")
except Exception as e:
    all_steps_passed = False
    print(f"Step 4 failed: {e}")

# Step 5: Test the ONNX model using onnxruntime
try:
    import onnxruntime as rt
    import numpy as np
    
    # Run inference with ONNX Runtime
    sess = rt.InferenceSession(onnx_model_path)
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]
    
    assert len(pred_onx) == len(X_test), "Inference result length does not match input length."
    
    print("Step 5: Testing the ONNX model using onnxruntime passed.")
except AssertionError as e:
    all_steps_passed = False
    print(f"Step 5 failed: {e}")
except Exception as e:
    all_steps_passed = False
    print(f"Step 5 failed: {e}")

# Final confirmation message
if all_steps_passed:
    print("All extensive tests for the 'skl2onnx' package completed successfully.")


Step 1: Importing skl2onnx and necessary modules passed.
Step 2: Training a scikit-learn RandomForestClassifier passed.
Step 3: Converting the model to ONNX format passed.
Step 4: Saving and validating the ONNX model passed.
Step 5: Testing the ONNX model using onnxruntime passed.
All extensive tests for the 'skl2onnx' package completed successfully.
